### スクレイピング

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import re

def preprocessing(data):
    """文字列内の1つの数字列を抽出し、成形して返す関数"""
    data_str = str(data)  # データを文字列に変換
    cleaned_data = re.sub(r"[^\d.-]", "", data_str)
    if cleaned_data and cleaned_data != "-":
        cleaned_data = float(cleaned_data)
        if "万円" in data_str:
            return cleaned_data * 10000
        else:
            return cleaned_data
    else:
        return 0


def floor_number(input_floor):
    """階数を正規表現で抽出して返す関数"""
    pattern = r'\d+'
    numbers = re.findall(pattern, input_floor)
    if numbers:
        return numbers[0]
    else:
        return input_floor

def maxfloor_number(input_maxfloor):
    """階数（階建）を正規表現で抽出して返す関数"""
    pattern = r'(\d+)階建'
    match = re.search(pattern, input_maxfloor)
    if match:
        return match.group(1)
    else:
        return input_maxfloor

def year_number(input_year):
    """築年数を正規表現で抽出して返す関数"""
    input_year = str(input_year)
    pattern = r'築(\d+)年'
    match = re.search(pattern, input_year)
    if match:
        return match.group(1)
    elif input_year == "新築":
        return 0
    else:
        return input_year

def scrape_suumo(REQUEST_URL):
    """スーモのスクレイピングを行い、データフレームを返す関数"""
    res = requests.get(REQUEST_URL)
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")
    maxpage = int(soup.select("ol.pagination-parts a")[-1].text)

    if res.status_code == 200:
        # 各データを格納するためのリスト
        names = []
        addresses = []
        station_info1_list = []
        station_info2_list = []
        station_info3_list = []
        years = []
        maxfloors = []

        floors = []
        rent_fees = []
        maintenance_fees = []
        deposit_fees = []
        gratuity_fees = []
        layouts = []
        areas = []

        # ページごとのスクレイピング
        for k in tqdm(range(int(maxpage))):
            res = requests.get(REQUEST_URL+"&page="+str(k))
            res.encoding = "utf-8"
            soup = BeautifulSoup(res.text, "html.parser")
            details = soup.select("div.cassetteitem-detail")
            time.sleep(1)

            # 各物件のデータを取得
            for i in range(len(details)):
                detail = details[i]
                name = detail.select("div.cassetteitem_content-title")[0].text
                address = detail.select("li.cassetteitem_detail-col1")[0].text
                station_info1 = detail.select("div.cassetteitem_detail-text")[0].text
                station_info2 = detail.select("div.cassetteitem_detail-text")[1].text
                station_info3 = detail.select("div.cassetteitem_detail-text")[2].text
                year, maxfloor = detail.select("li.cassetteitem_detail-col3")[0].text.split()

                items = soup.select("div.cassetteitem-item")[i].select("tbody")
                for j in range(len(items)):
                    item = items[j]
                    floor = item.select("td")[2].text.split()[0]
                    rent_fee, maintenance_fee = item.select("td")[3].text.split()
                    deposit_fee, gratuity_fee = item.select("td")[4].text.split()
                    layout, area = item.select("td")[5].text.split()

                    year = year_number(year)
                    maxfloor = maxfloor_number(maxfloor)
                    floor = floor_number(floor)
                    floor = preprocessing(floor)
                    rent_fee = preprocessing(rent_fee)
                    maintenance_fee = preprocessing(maintenance_fee)
                    deposit_fee = preprocessing(deposit_fee)
                    gratuity_fee = preprocessing(gratuity_fee)
                    area = preprocessing(area)

                    names.append(name)
                    addresses.append(address)
                    station_info1_list.append(station_info1)
                    station_info2_list.append(station_info2)
                    station_info3_list.append(station_info3)
                    years.append(year)
                    maxfloors.append(maxfloor)
                    floors.append(floor)
                    rent_fees.append(rent_fee)
                    maintenance_fees.append(maintenance_fee)
                    deposit_fees.append(deposit_fee)
                    gratuity_fees.append(gratuity_fee)
                    layouts.append(layout)
                    areas.append(area)

        data = {
            'Name': names,
            'Address': addresses,
            'Station_Info1': station_info1_list,
            'Station_Info2': station_info2_list,
            'Station_Info3': station_info3_list,
            'Year': years,
            'Max_Floor': maxfloors,
            'Floor': floors,
            'Rent_Fee': rent_fees,
            'Maintenance_Fee': maintenance_fees,
            'Deposit_Fee': deposit_fees,
            'Gratuity_Fee': gratuity_fees,
            'Layout': layouts,
            'Area': areas
        }
        df_suumo = pd.DataFrame(data)
        return df_suumo
    else:
        print("res.status_codeエラー:",res.status_code)

def scrape_airdoor(REQUEST_URL):
    """エアドアのスクレイピングを行い、データフレームを返す関数"""
    res = requests.get(REQUEST_URL)
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")
    maxpage = int(soup.select("div.Search_resultSummary___Z5jC li a")[-1].text)

    if res.status_code == 200:
        # 各データを格納するためのリスト
        names = []
        addresses = []
        station_info1_list = []
        station_info2_list = []
        station_info3_list = []
        years = []
        maxfloors = []

        floors = []
        rent_fees = []
        maintenance_fees = []
        deposit_fees = []
        gratuity_fees = []
        layouts = []
        areas = []

        for k in tqdm(range(int(maxpage))):
            res = requests.get(REQUEST_URL+"&p="+str(k))
            res.encoding = "utf-8"
            soup = BeautifulSoup(res.text, "html.parser")
            time.sleep(1)

            if res.status_code == 200:
                propertyPanels = soup.select("div.PropertyPanel_propertyPanel__8oJ13")
                for i in range(len(propertyPanels)):
                    propertyPanel = propertyPanels[i]
                    name = propertyPanel.select("div.PropertyPanelBuilding_buildingTitle__tuPqN")[0].text
                    address = propertyPanel.select("p.is-mt5")[0].text
                    station_info1 = propertyPanel.select("div.PropertyPanelBuilding_buildingInformationSection__deSLp")[0].select("p")[0].text
                    station_info2 = soup.select("div.PropertyPanelBuilding_buildingInformationSection__deSLp")[0].select("p")[1].text
                    station_info3 = ""
                    year = soup.select("div.PropertyPanelBuilding_buildingInformationSection__deSLp")[1].select("p")[0].text
                    maxfloor = soup.select("div.PropertyPanelBuilding_buildingInformationSection__deSLp")[1].select("p")[1].text

                    if '新築' in year:
                        year = 1
                    else:
                        pattern = r'築([0-9]+)年'
                        result = re.search(pattern, year)
                        year = result.group(1)

                    roomitems = propertyPanel.select("a.PropertyPanelRoom_roomItem__95jRr")
                    for j in range(len(roomitems)):
                        roomitem = roomitems[j]

                        deposit_fee, gratuity_fee = preprocessing(roomitem.select("div.PropertyPanelRoom_initialPrices__d90C3")[0].select("li")[0].text), preprocessing(roomitem.select("div.PropertyPanelRoom_initialPrices__d90C3")[0].select("li")[1].text)

                        roomitemrightcontent = roomitem.select("span.is-ml5")[0].text
                        pattern = r'(\d+)[A-Za-z]*号室.*? / (\w+) / ([0-9.]+)㎡ / (.+)'
                        result = re.search(pattern, roomitemrightcontent)

                        if result:
                            room_number = result.group(1)
                            layout = result.group(2)
                            area = result.group(3)

                            room_number = room_number.translate(str.maketrans('０１２３４５６７８９', '0123456789'))
                            if '0' in room_number:
                                zero_position = len(room_number) - room_number[::-1].index('0') - 1
                                if zero_position:
                                    floor = room_number[:zero_position]
                                else:
                                    for digit in room_number:
                                        if digit != '0':
                                            floor = digit
                            else:
                                floor = room_number[0]
                            floor = int(floor)
                        else:
                            floor = 1

                        rentprices = roomitem.select("div.PropertyPanelRoom_rentPrice__XdPUp")[0].text
                        pattern = r'([0-9,]+)円 \(管理費([0-9,]+)円\)'
                        result = re.search(pattern, rentprices)
                        rent_fee = result.group(1)
                        maintenance_fee = result.group(2)
                        rent_fee = int(rent_fee.replace(',', ''))
                        maintenance_fee = int(maintenance_fee.replace(',', ''))

                        maxfloor = maxfloor_number(maxfloor)
                        deposit_fee = preprocessing(deposit_fee)
                        gratuity_fee = preprocessing(gratuity_fee)

                        names.append(name)
                        addresses.append(address)
                        station_info1_list.append(station_info1)
                        station_info2_list.append(station_info2)
                        station_info3_list.append(station_info3)
                        years.append(year)
                        maxfloors.append(maxfloor)
                        floors.append(floor)
                        rent_fees.append(rent_fee)
                        maintenance_fees.append(maintenance_fee)
                        deposit_fees.append(deposit_fee)
                        gratuity_fees.append(gratuity_fee)
                        layouts.append(layout)
                        areas.append(area)

        data = {
            'Name': names,
            'Address': addresses,
            'Station_Info1': station_info1_list,
            'Station_Info2': station_info2_list,
            'Station_Info3': station_info3_list,
            'Year': years,
            'Max_Floor': maxfloors,
            'Floor': floors,
            'Rent_Fee': rent_fees,
            'Maintenance_Fee': maintenance_fees,
            'Deposit_Fee': deposit_fees,
            'Gratuity_Fee': gratuity_fees,
            'Layout': layouts,
            'Area': areas
        }
        df_airdoor = pd.DataFrame(data)
        return df_airdoor
    else:
        print("res.status_codeエラー:",res.status_code)


# スクレイピングを行うURLの指定
suumo_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&ta=13&bs=040&ekInput=28860&tj=30&nk=-1&ekInput=17460&tj=30&nk=-1&ct=25.0&cb=0.0&et=9999999&mt=9999999&mb=0&cn=15&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&pc=30"
airdoor_url = "https://airdoor.jp/list?st=d-91224-5779076&uf=10"

# スーモとエアドアのデータをそれぞれ取得
df_airdoor = scrape_airdoor(airdoor_url)
# df_suumo = scrape_suumo(suumo_url)

# # スクレイピング結果を結合して全体のデータフレームを作成
# df_all = pd.concat([df_suumo, df_airdoor], ignore_index=True)
# #'Year','Floor','Rent_Fee','Maintenance_Fee','Deposit_Fee','Gratuity_Fee','Layout'が一致するデータを統合
# df_all=df_all[df_all.duplicated(subset=['Year','Floor','Rent_Fee','Maintenance_Fee','Deposit_Fee','Gratuity_Fee','Layout'], keep=False)]


  0%|          | 0/3 [00:01<?, ?it/s]


IndexError: list index out of range

### スプレッドシートへの転記

In [10]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from dotenv import load_dotenv
import os

#.envファイルから環境変数を取得
load_dotenv()
SERVICE_ACCOUNT_FILE = os.getenv('SUUMO_DB_gspread_json_filepath')
SPERADSHEET_KEY = os.getenv('SUUMO_DB_gspread_key')

# GoogleSheetsAPI、GoogleDriveAPI、及び認証鍵の指定
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Service Accountの認証情報を取得
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE,SCOPES)

# 認証情報を用いてGoogleSheetsにアクセス
gs = gspread.authorize(credentials)

# 対象のスプレッドシートとワークシートを指定
workbook=gs.open_by_key(SPERADSHEET_KEY)
worksheet=workbook.worksheet("元データ")

#dfから値を習得
values = [df_all.columns.values.tolist()] + df_all.values.tolist()

# ワークシートの指定したセル(B2)から値を追加
worksheet.update("B2", values)

C:\Users\zk-ht\AppData\Local\Temp\ipykernel_11800\3242504353.py:29: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("B2", values)


{'spreadsheetId': '1E5sSVGEaVJvmI-gU48-6glk5WtS7wq0nPL6i5FDPd5A',
 'updatedRange': "'元データ'!B2:O27033",
 'updatedRows': 27032,
 'updatedColumns': 14,
 'updatedCells': 378448}